<a href="https://www.spe.org/events/en/2022/conference/22apog/asia-pacific-oil-and-gas-conference-and-exhibition.html"><img src = "https://www.spe.org/binaries/content/gallery/specms/speevents/organization-logos/spe-logo-2020.png" width = 200> 

<h1 align=center><font size = 5>Prediction of Recovery Factor using Machine Learning Methods</font></h1>

<h1 align=center><font size = 4> Munish Kumar, Kannapan Swaminathan</font></h1>
<h1 align=center><font size = 4> Part 5: Prediction of Recovery Factor</font></h1>
<h1 align=center><font size = 3> ERCE 2022 </font></h1>

#### Libraries

In [1]:
# Only install the following libraries if you dont have it, otherwise leave it commented out

#!conda install -c anaconda natsort --yes
#!conda install -c anaconda xlrd --yes
#!pip install pycaret[full] --user
#!pip install mlflow --user
#!pip install tune-sklearn ray[tune] --user
#!pip install optuna -- user
#!pip install hyperopt --user

# General Libraries
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import NullFormatter
import time
import re
import requests
import pickle
import seaborn as sns
import os
import glob
import sys
from natsort import natsorted
sns.set()

import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

# Sklearn Liraries
from sklearn import preprocessing

import datetime
from datetime import timedelta, date 
start = time.time()
%matplotlib inline

import ray
from ray import tune

# Forces the print statement to show everything and not truncate
# np.set_printoptions(threshold=sys.maxsize) 
print('Libraries imported')

Libraries imported


In [2]:
# Function to read out file
def file_out(df, dir_name, base_filename, filename_suffix):
    try:
        csvs_sht = os.path.join(dir_name, base_filename + "." + filename_suffix)
        df.to_csv(csvs_sht, index = False, header=True)
        print ("Final File Extract Produced")
    except IOError:
        print("I/O error")
    return

###### Declare some global variables

In [3]:
#Input Data
#dir_name = r'C:\Users\kswaminathan\OneDrive\01_KannaLibrary\15_Analogs'
dir_name = r'C:\Users\mkumar\Documents\GitHub\SPE_Paper\Final'
filename_suffix = 'csv'

##### Read in the data 

In [4]:
skiprows = 0
dfblind = pd.read_excel("BlindTest_SSOIL_MOCK.xlsx", sheet_name='Inputs', thousands=',', skiprows = skiprows)
#dfblind = dfblind.loc[:, ~df.columns.str.contains('^Unnamed')] 
dfblind_ori = dfblind.copy()
dfblind.drop('307_Recovery factor (ultimate oil) (%)', axis=1, inplace=True)
dfblind.dropna(axis = 0, inplace=True)

dfblind.head(10)

,10_Region,11_Basin name,52_Well spacing (average) (ac),72_Onshore or offshore,75_Temperature (original) (deg. F),77_Pressure (original) (psi),82_Drive mechanism (main),117_Reservoir top subsea depth (ft TVDSS),149_Reservoir period,153_Depositional system (main),161_Reservoir architecture (main),176_Net/gross ratio (average),204_Porosity (matrix average) (%),210_Permeability (air average) (mD),218_API gravity (average) (deg. API),225_Viscosity (oil average) (cp),252_GOR (initial average) (SCF/STB),293_Original in-place oil equivalent (MMBOE),EORIOR
0,ASIA-PACIFIC,AMADEUS,16,ONSHORE,146.000000,1805.000000,GAS DRIVE,1000,ORDOVICIAN/CAMBRIAN,24:COASTAL,LAYER-CAKE,0.46,8.0,10,49.0,0.360000,850,498,4
1,LATIN AMERICA,MARACAIBO,90,ONSHORE,180.000000,3450.000000,GAS DRIVE,4200,PALEOGENE,24:COASTAL,JIGSAW-PUZZLE,0.50,24.6,470,10.5,344.000000,108,36840,7
2,ASIA-PACIFIC,SABAH,0,OFFSHORE,115.677306,616.262767,GAS DRIVE,1000,NEOGENE,24:COASTAL,JIGSAW-PUZZLE,0.30,25.0,220,32.0,3.648688,650,549,4
3,LATIN AMERICA,MARACAIBO,33,OFFSHORE,120.000000,1198.000000,COMPACTION,1400,NEOGENE,24:COASTAL,JIGSAW-PUZZLE,0.35,24.0,530,17.0,93.000000,598,3636,6
4,ASIA-PACIFIC,NW BORNEO-SABAH,0,OFFSHORE,129.000000,1102.000000,GAS DRIVE,1640,NEOGENE,24:COASTAL,LAYER-CAKE,0.50,25.0,1500,22.0,5.000000,496,2684,7
5,AFRICA,GULF OF SUEZ,27,OFFSHORE,265.000000,5506.000000,AQUIFER DRIVE,10578,CRETACEOUS/PALEOZOIC,21:FLUVIAL,LAYER-CAKE,0.20,18.0,450,27.0,1.010000,279,1644,6
6,ASIA-PACIFIC,BOHAI,20,ONSHORE,171.130601,3244.000000,GAS DRIVE,5426,PALEOGENE,23:LACUSTRINE,LAYER-CAKE,0.30,18.0,1800,33.5,3.500000,620,605,7


## Load the Model

In [5]:
from pycaret.classification import load_model
from pycaret.classification import predict_model

from functools import reduce

final_blend = load_model('Blended_model_21042022')
final_db = load_model('Boosted_CatBoost_21042022')
final_gbr = load_model('Boosted_GBR_21042022')
final_rf = load_model('Boosted_RF_21042022')

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [7]:
predict_model(final_blend, data=dfblind, round=2)

,10_Region,11_Basin name,52_Well spacing (average) (ac),72_Onshore or offshore,75_Temperature (original) (deg. F),77_Pressure (original) (psi),82_Drive mechanism (main),117_Reservoir top subsea depth (ft TVDSS),149_Reservoir period,153_Depositional system (main),161_Reservoir architecture (main),176_Net/gross ratio (average),204_Porosity (matrix average) (%),210_Permeability (air average) (mD),218_API gravity (average) (deg. API),225_Viscosity (oil average) (cp),252_GOR (initial average) (SCF/STB),293_Original in-place oil equivalent (MMBOE),EORIOR,Label
0,ASIA-PACIFIC,AMADEUS,16,ONSHORE,146.000000,1805.000000,GAS DRIVE,1000,ORDOVICIAN/CAMBRIAN,24:COASTAL,LAYER-CAKE,0.46,8.0,10,49.0,0.360000,850,498,4,7
1,LATIN AMERICA,MARACAIBO,90,ONSHORE,180.000000,3450.000000,GAS DRIVE,4200,PALEOGENE,24:COASTAL,JIGSAW-PUZZLE,0.50,24.6,470,10.5,344.000000,108,36840,7,7
2,ASIA-PACIFIC,SABAH,0,OFFSHORE,115.677306,616.262767,GAS DRIVE,1000,NEOGENE,24:COASTAL,JIGSAW-PUZZLE,0.30,25.0,220,32.0,3.648688,650,549,4,11
3,LATIN AMERICA,MARACAIBO,33,OFFSHORE,120.000000,1198.000000,COMPACTION,1400,NEOGENE,24:COASTAL,JIGSAW-PUZZLE,0.35,24.0,530,17.0,93.000000,598,3636,6,14
4,ASIA-PACIFIC,NW BORNEO-SABAH,0,OFFSHORE,129.000000,1102.000000,GAS DRIVE,1640,NEOGENE,24:COASTAL,LAYER-CAKE,0.50,25.0,1500,22.0,5.000000,496,2684,7,29
5,AFRICA,GULF OF SUEZ,27,OFFSHORE,265.000000,5506.000000,AQUIFER DRIVE,10578,CRETACEOUS/PALEOZOIC,21:FLUVIAL,LAYER-CAKE,0.20,18.0,450,27.0,1.010000,279,1644,6,30
6,ASIA-PACIFIC,BOHAI,20,ONSHORE,171.130601,3244.000000,GAS DRIVE,5426,PALEOGENE,23:LACUSTRINE,LAYER-CAKE,0.30,18.0,1800,33.5,3.500000,620,605,7,40


In [ ]:
BlindPredict1 = predict_model(final_blend, data=dfblind, round=2)
BlindPredict2 = predict_model(final_db, data=dfblind, round=2)
BlindPredict3 = predict_model(final_gbr, data=dfblind, round=2)
BlindPredict4 = predict_model(final_rf, data=dfblind, round=2)

BlindPredict1 = BlindPredict1.rename(columns={'Label': 'Blended Predicted Recovery factor (ultimate oil) (%)'
                                 })
BlindPredict2 = BlindPredict2.rename(columns={'Label': 'CatBoost Predicted Recovery factor (ultimate oil) (%)'
                                 })
BlindPredict3 = BlindPredict3.rename(columns={'Label': 'Gradient Boost Predicted Recovery factor (ultimate oil) (%)'
                                 })
BlindPredict4 = BlindPredict4.rename(columns={'Label': 'Random Forest Predicted Recovery factor (ultimate oil) (%)'
                                 })

BlindPredict_df = [BlindPredict1, BlindPredict2, BlindPredict3, BlindPredict4]

BlindPredict = pd.concat(BlindPredict_df, axis = 0, join = 'outer').drop_duplicates()
BlindPredict = BlindPredict.reset_index(drop=True)

BlindPredict

In [ ]:
aggregation_functions = {
    'Blended Predicted Recovery factor (ultimate oil) (%)': 'sum', 
    'CatBoost Predicted Recovery factor (ultimate oil) (%)': 'sum',
    'Gradient Boost Predicted Recovery factor (ultimate oil) (%)': 'sum',
    'Random Forest Predicted Recovery factor (ultimate oil) (%)': 'sum',
}

labels = dfblind.columns.tolist()

BlindPredict_new = BlindPredict.groupby(labels).aggregate(aggregation_functions)

file_out(BlindPredict, dir_name, 'Blind_Test_Out' , filename_suffix)

BlindPredict_new

#### New Model

In [ ]:
skiprows = 0
dfblind1 = pd.read_excel("BlindTest_SSOIL.xlsx", sheet_name='Inputs', thousands=',', skiprows = skiprows)
#dfblind = dfblind.loc[:, ~df.columns.str.contains('^Unnamed')] 
dfblind1.drop('307_Recovery factor (ultimate oil) (%)', axis=1, inplace=True)
dfblind1.dropna(axis = 0, inplace=True)

dfblind1.head()

In [ ]:
#dfblind1.drop('118_Structural flank dip (average) (deg.)', axis=1, inplace=True)
#dfblind1['176_Net/gross ratio (average)']=dfblind1['176_Net/gross ratio (average)']*100
#dfblind1['204_Porosity (matrix average) (%)']=dfblind1['204_Porosity (matrix average) (%)']*100
#dfblind1["149_Reservoir period"] = dfblind1["149_Reservoir period"].str.replace('Cretaceous', 'CRETACEOUS')
#dfblind1["11_Basin name"] = dfblind1["11_Basin name"].str.replace('Siberien Western', 'SIBERIAN WESTERN')


BlindPredict1 = predict_model(final_blend, data=dfblind1, round=2)
BlindPredict2 = predict_model(final_db, data=dfblind1, round=2)
BlindPredict3 = predict_model(final_gbr, data=dfblind1, round=2)
BlindPredict4 = predict_model(final_rf, data=dfblind1, round=2)

BlindPredict1 = BlindPredict1.rename(columns={'Label': 'Blended Predicted Recovery factor (ultimate oil) (%)'
                                 })
BlindPredict2 = BlindPredict2.rename(columns={'Label': 'CatBoost Predicted Recovery factor (ultimate oil) (%)'
                                 })
BlindPredict3 = BlindPredict3.rename(columns={'Label': 'Gradient Boost Predicted Recovery factor (ultimate oil) (%)'
                                 })
BlindPredict4 = BlindPredict4.rename(columns={'Label': 'Random Forest Predicted Recovery factor (ultimate oil) (%)'
                                 })

BlindPredict_df = [BlindPredict1, BlindPredict2, BlindPredict3, BlindPredict4]

BlindPredict = pd.concat(BlindPredict_df, axis = 0, join = 'outer').drop_duplicates()
BlindPredict = BlindPredict.reset_index(drop=True)

BlindPredict

In [ ]:
aggregation_functions = {
    'Blended Predicted Recovery factor (ultimate oil) (%)': 'sum', 
    'CatBoost Predicted Recovery factor (ultimate oil) (%)': 'sum',
    'Gradient Boost Predicted Recovery factor (ultimate oil) (%)': 'sum',
    'Random Forest Predicted Recovery factor (ultimate oil) (%)': 'sum',
}

labels = dfblind1.columns.tolist()

BlindPredict_new = BlindPredict.groupby(labels).aggregate(aggregation_functions)

file_out(BlindPredict, dir_name, 'Blind_test_Out_mexico' , filename_suffix)

BlindPredict_new